In [1]:
import pandas as pd
import regex as re
import os

## Testing

In [3]:
df = pd.read_csv('data/full_transcripts_cleaned.csv')
# df['col'].str.split().str.join(' ')
df.head()

,Unnamed: 0,series,episode,line,char,text
0,1,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,2,ADJUSTED,"Hello, one and all, and welcome back to anoth..."
1,2,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,3,ADJUSTED,"Heroes of Pure Heart."" She's also a dungeon m..."
2,3,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,4,Kelly,"Yeah, this is big, woo, the crowd goes wild. ..."
3,4,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,5,Brennan,"Flowers, (laughs) flowers, flowers. Kelly, it..."
4,5,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,6,Kelly,"Thanks for having me, Brennan. It's really ex..."


In [4]:
dm_token = '<|dm|>'
player_token = '<|player|>'
session_token = '<|sesh|>'
eos_token = '<|endoftext|>'
bos_token =  '<|startoftext|>'


## Save D20 Data

In [5]:
episode = ''
full_text = ''
player_index = 0
for index, row in df.iterrows():
    if episode != row.episode:
        episode = row.episode
        player_index = 0
        player_mapping = {
            'matt':dm_token,
            'brennan':dm_token,
            'matt':dm_token,
            'adjusted':f'0 {player_token}',
        }
        full_text += '\n'
        full_text += session_token
    if row.char.lower() not in  player_mapping.keys():
        player_mapping[row.char.lower()] = f'{player_index} {player_token}'
        player_index += 1
    line = row.text
    line.replace('\xa0', '')
    # line.replace('\\', '')
    full_text += f' {player_mapping[row.char.lower()]} {bos_token} {line} {eos_token}\n'


In [6]:
print(full_text[:5000])


<|sesh|> 0 <|player|> <|startoftext|>  Hello, one and all, and welcome back to another exciting episode of "Adventuring Academy," Dimension 20's vodcast, where we talk about all things gaming and tabletop, and how to play, and what different theories and practices work the best at your table and tables all over. Thank you so much for tuning in. Oh my gosh, everybody, today I am so excited for the incredible guest who's gonna be here talking with us about all things tabletop. You know her work. She is an animation TV writer for "My Little Pony <|endoftext|>
 0 <|player|> <|startoftext|>  Heroes of Pure Heart." She's also a dungeon master. You've seen her dungeon mastering on "Girls Guts Glory," which is streamed from Wizards of the Coast. And you also can find her musical that she wrote and directed, "STARRY," the musical, available on Spotify right now, as well as an upcoming project, which is gonna be incredible and D&D, an all-native D&D actual play show coming soon to a live stream

In [7]:
with open('data/transcript_train.txt', 'w') as file:
    file.writelines(full_text)

## Save Adventure Zone Data

In [79]:
file_dir = 'data/adventure_zone/files/'
dest_dir = 'data/adventure_zone/'
data = []

for filename in os.listdir(file_dir):
    text = open(file_dir+filename, 'r').read()  
    rows = re.split(':', text)
    for i in range(len(rows)):
        row = rows[i]
        row = re.split('\n',row)
        data.append(['adventure zone',filename, i,row[-1], ''.join(row[:-1])])

df2 = pd.DataFrame(data, columns=['series','episode', 'line', 'char', 'text'])
df2.info()
df2.char = df2.char.shift(1)
df2.drop(0,axis=0, inplace=True)
df2.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139431 entries, 0 to 139430
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   series   139431 non-null  object
 1   episode  139431 non-null  object
 2   line     139431 non-null  int64 
 3   char     139431 non-null  object
 4   text     139431 non-null  object
dtypes: int64(1), object(4)
memory usage: 5.3+ MB


,series,episode,line,char,text
139426,adventure zone,TAZ227-Ethersea40-Episode-35.txt,759,Griffin,What usually happens with you when you cast ...
139427,adventure zone,TAZ227-Ethersea40-Episode-35.txt,760,Clint,Usually can se nd like an image into the bea...
139428,adventure zone,TAZ227-Ethersea40-Episode-35.txt,761,Griffin,Okay. Something different happens this time.
139429,adventure zone,TAZ227-Ethersea40-Episode-35.txt,762,Clint,Okay.
139430,adventure zone,TAZ227-Ethersea40-Episode-35.txt,763,Griffin,Your mind is flooded with… a language that d...


## Save Critical Role Data

In [80]:
file_dir = 'data/critical_role/files/'
dest_dir = 'data/critical_role/'
data = []

for filename in os.listdir(file_dir):
    text = open(file_dir+filename, 'r').read()  
    if text.find('<Part I>'):
        text = ''.join(text.split('<Part I>')[1:])
    rows = re.split(':', text)
    for i in range(len(rows)):
        row = rows[i]
        row = re.split('\n',row)
        data.append(['critical role',filename, i,row[-1], ''.join(row[:-1])])

df3 = pd.DataFrame(data, columns=['series','episode', 'line', 'char', 'text'])
df3.info()
df3.char = df3.char.shift(1)
df3.drop(0,axis=0, inplace=True)
df3.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937447 entries, 0 to 937446
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   series   937447 non-null  object
 1   episode  937447 non-null  object
 2   line     937447 non-null  int64 
 3   char     937447 non-null  object
 4   text     937447 non-null  object
dtypes: int64(1), object(4)
memory usage: 35.8+ MB


,series,episode,line,char,text
937442,critical role,Frigid_Propositions.txt,2507,MARISHA,Wow.
937443,critical role,Frigid_Propositions.txt,2508,LIAM,"Me too, me too. Eiselcross."
937444,critical role,Frigid_Propositions.txt,2509,MARISHA,Eiselcross!
937445,critical role,Frigid_Propositions.txt,2510,SAM,"Oh, man."
937446,critical role,Frigid_Propositions.txt,2511,MATT,"This is going to be different. All righty, we..."


In [81]:
df = pd.concat([df1, df2, df3])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1395159 entries, 1 to 937446
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   series   1395159 non-null  object
 1   episode  1395159 non-null  object
 2   line     1395159 non-null  int64 
 3   char     1395159 non-null  object
 4   text     1395159 non-null  object
dtypes: int64(1), object(4)
memory usage: 63.9+ MB


In [88]:
df.to_csv('data/full_transcripts_cleaned.csv')

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 935818 entries, 0 to 937446
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   series   935818 non-null  object
 1   episode  935818 non-null  object
 2   line     935818 non-null  int64 
 3   char     935818 non-null  object
 4   text     935818 non-null  object
dtypes: int64(1), object(4)
memory usage: 42.8+ MB


In [23]:
# temp = df.groupby('char').count()
temp = df.groupby(['char']).count().sort_values('series')
# temp[temp['series']>1]
# df[df.char == "Brennan"]

In [68]:
# data = df.values.tolist()

In [82]:
data[0]

['critical role', 'The_Echo_Tree.txt', 0, 'MATT', '(laughing)']

In [84]:
for i in range(len(data)):
    if len(data[i][3]) > 20:
        data[i][4] = data[i][4] + data[i][3]
        data[i][3] = 'ADJUSTED'
    elif len(data[i][3]) <= 1:
        data[i][3] = "BLANK"
    if len(data[i][4]) ==0:
        data[i][4] = pd.NA

In [85]:
df.dropna(axis=0)

,series,episode,line,char,text
1,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,1,Brennan,
2,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,2,"Hello, one and all, and welcome back to anoth...",
3,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,3,"Equestria Girls,"" ""Final Space,"" and ""Mao Mao","Heroes of Pure Heart."" She's also a dungeon m..."
4,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,4,Kelly,"Yeah, this is big, woo, the crowd goes wild. ..."
5,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,5,Brennan,"Flowers, (laughs) flowers, flowers. Kelly, it..."
...,...,...,...,...,...
937442,critical role,Frigid_Propositions.txt,2507,MARISHA,Wow.
937443,critical role,Frigid_Propositions.txt,2508,LIAM,"Me too, me too. Eiselcross."
937444,critical role,Frigid_Propositions.txt,2509,MARISHA,Eiselcross!
937445,critical role,Frigid_Propositions.txt,2510,SAM,"Oh, man."


In [86]:
df = pd.DataFrame(data, columns=['series','episode', 'line', 'char', 'text'])
df.dropna(axis=0, inplace=True)
# df.info()
# df.char = df.char.shift(1)
# df.drop(0,axis=0, inplace=True)

df.head()

,series,episode,line,char,text
0,critical role,The_Echo_Tree.txt,0,MATT,(laughing)
1,critical role,The_Echo_Tree.txt,1,LAURA,"And welcome back to tonight's episode. So, pi..."
2,critical role,The_Echo_Tree.txt,2,MATT,Yeah.
3,critical role,The_Echo_Tree.txt,3,LAURA,"It's been a long week. However, you guys have..."
4,critical role,The_Echo_Tree.txt,4,MATT,"Ahh, totally."


In [43]:
temp[temp.series > 10].sum()

series     1378604
episode    1378604
line       1378604
text       1378604
dtype: int64

In [58]:
df[df.char ==' we get a bunch of devils, and they get to install two in a place of divine worship to be determined. Third pact is']

,series,episode,line,char,text


In [59]:
df[df.text =='']

,series,episode,line,char,text
2069,dimension 20,The_Chosen_Ones.txt,188,Jammer,
4112,dimension 20,Ambush_on_the_Sucrosi_Road.txt,416,Emily,
11955,dimension 20,My_Green_Heaven.txt,587,Emily,
13670,dimension 20,The_Two_Balls.txt,4,ADJUSTED,
15045,dimension 20,The_Sisterly_Showdown.txt,355,Murph,
...,...,...,...,...,...
1391816,critical role,Rebirth.txt,2739,MARISHA,
1392514,critical role,Rebirth.txt,3437,TRAVIS,
1392589,critical role,Rebirth.txt,3512,MATT,
1392650,critical role,Frigid_Propositions.txt,0,,


In [54]:
mask = (df['char'].str.len() > 50)
df[mask]

,series,episode,line,char,text
2,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,2,"Hello, one and all, and welcome back to anoth...",
16,dimension 20,The_Joy_of_Mistakes_(with_Kelly_Lynne_D%27Ange...,16,But when I was... I just love YouTube. And I ...,The Last Airbender.'? Are you talking about '...
436,dimension 20,"Oh_No,_I%E2%80%99m_In_Love_With_Chupacabra.txt",85,"Yeah, I had to... So like we had someone like...","50 stapler, take the stapler. The staples are ..."
1921,dimension 20,The_Chosen_Ones.txt,40,"Perfect. We love asking questions, so feel fr...",00 p.m.?
3014,dimension 20,The_Chosen_Ones.txt,1133,We're in Greenwich Mean Time. I take... If Gr...,"00 a.m. here, something like that?"
...,...,...,...,...,...
1392066,critical role,Rebirth.txt,2989,"Finally, ♪ it has happened to me ♪ ♪ and I'm ...",♪ Molly! You gave your life to save us! Why a...
1392149,critical role,Rebirth.txt,3072,"I'll feel the Slow drop from my form, and I'l...",Are you sure you don't want to hang a tie on ...
1392201,critical role,Rebirth.txt,3124,And I will then wave my hand toward Essek and...,Wake up! And cast Dispel at fifth.
1393515,critical role,Frigid_Propositions.txt,865,He goes ahead and stealthily takes off (laugh...,Oh. This is a different Yeza.


In [45]:
temp.index

Index([' You glance inside, and you can see what appear to be piles of goods',
       ' walking up into this swamp. Second mistake',
       ' wards and shields, and also metamagic',
       ' we forgot to mention that next week during Game the Game, thanks to Magic Puzzle Quest, Matt and Marisha, Taliesin, Liam, and myself will be joining Ivan and Becca for Game the Game, from 5',
       ' we get a bunch of devils, and they get to install two in a place of divine worship to be determined. Third pact is',
       ' we return to Exandria in an age long past, in the flying city of Avalir. I would describe the place that we return to in Avalir, but it might be rude of me because it doesn't belong to me. So I might ask for someone else at the table to describe it, but I will say this',
       ' we're going to go through these doors. There's going to be three tripwires. That's what they're called',
       ' what are the reasons that we have to prioritize this over something else? Because the t